#### Objective function for Minimum Probability Flow
Given some observations $\mathcal{D}$, we can estimate the parameter using maximum likelihood which is also equivalent to minimizing the KL divergence between the model distribution $\mathbf{p}^{(0)}$ and the model distribution $\mathbf{p}^{(\infty)}$

$$\hat{\theta}_{ML} = argmin_{\theta}~ D_{KL}\left(\mathbf{p}^{(0)}||\mathbf{p}^{(\infty)}(\theta)\right)$$

In MPF, instead of running the dynamics for infinite time, we minimize the KL divergence after running the dynamics for infinitesimal time $\epsilon$,

$$\hat{\theta}_{MPF} = argmin_{\theta}~K(\theta) $$
where $$K(\theta) = D_{KL}\left(\mathbf{p}^{(0)}||\mathbf{p}^{(\epsilon)}(\theta)\right)$$

Since $\epsilon$ is taken to be small, we can perform first order [Taylor expansion](https://en.wikipedia.org/wiki/Taylor_series),

$$K(\theta) \approx D_{KL}\left(\mathbf{p}^{(0)}||\mathbf{p}^{(t)}(\theta)\right)\Big|_{t=0} + \epsilon \frac{\partial D_{KL}\left(\mathbf{p}^{(0)}||\mathbf{p}^{(t)}(\theta)\right)}{\partial t}\Bigg|_{t=0}$$

The first term is zero so we evaluate the derivative in the second term,

$$\frac{\partial D_{KL}\left(\mathbf{p}^{(0)}||\mathbf{p}^{(t)}(\theta)\right)}{\partial t}\Bigg|_{t=0} = -\sum_{x \in \mathcal{D}}p^{(0)}(x)\frac{\partial}{\partial t}\log p^{(t)}(x|\theta)\Bigg|_{t=0}$$

$$ = -\sum_{x \in \mathcal{D}}p^{(0)}(x)\frac{\dot{p}^{(t)}(x|\theta)}{p^{(t)}(x|\theta)}\Bigg|_{t=0}$$

$$ = -\sum_{x \in \mathcal{D}}\dot{p}^{(0)}(x|\theta)$$

$$ = -\sum_{x \in \mathcal{D}}\left(\sum_{\substack{y \in \mathcal{X}\\y \neq x}}\Gamma_{xy}\,p^{(0)}(y)-\sum_{\substack{y \in \mathcal{X}\\y \neq x}}\Gamma_{yx}\,p^{(0)}(x)\right)$$

## Still not completed as I don't understand some parts

#### Boltzmann Machine network
We consider the Boltzmann Machine where the connectivity of the different states is given to be $g_{xy} \neq 0$ if and only if $x$ and $y$ differ only at exactly one bit location. We also think of $W$ to be **symmetric** with the diagonals thought of as the biases. Previously, we set the diagonals to be zeros with the biases computed separately. With this new implementation, the parameters and biases are stored in a single matrix $W$.

Suppose that $x$ differs from $y$ at the $h$ location, and let $y_{-h}$ denote the elements of $y$ except the one at the $h$ location,

$$E(y|\theta) - E(x|\theta) = E(y_h,y_{-h}|\theta) - E(1-y_h,y_{-h}|\theta)$$

$$=-\frac{1}{2}y^\top \theta y + \frac{1}{2}x^\top \theta x$$

we can do a simple expansion which gives 

$$ \frac{1}{2}(x^\top \theta x -x^\top \theta y + x^\top \theta y - y^\top \theta y )$$

after some simplification we get

$$\frac{1}{2}a^\top \theta b$$

where $a = x + y$ and $b = x -y$. Writing out $x$ and $y$ explicitly,

$$y=\begin{pmatrix}y_1\\ \vdots\\ y_h\\ \vdots\\ y_n\end{pmatrix} \quad x = \begin{pmatrix}y_1\\ \vdots\\1-y_h\\ \vdots\\ y_n\end{pmatrix}$$

Solving for the $a$ and $b$, we have

$$a=\begin{pmatrix}2y_1\\ \vdots\\ 1\\ \vdots\\ 2y_n\end{pmatrix} \quad b = \begin{pmatrix}0\\ \vdots\\1-2y_h\\ \vdots\\ 0\end{pmatrix}$$

thus

$$\frac{1}{2}E(y|\theta) - E(x|\theta) = \frac{1}{4}a^\top \theta b = \begin{pmatrix}y_1(1/2-y_h)\theta_{1h}, \ldots,1/2(1/2-y_h)\theta_{hh}\ldots,y_n(1/2-x_h)\theta_{nh} \end{pmatrix}$$

We can then factorize the common $1/2 - y_h$ term out and letting $\delta = 1/2 - y$, with $*$ denoting element-wise product, we have

$$(1/2-y_h)*((\theta y)_h - \theta_{hh}(1/2-y_h))$$

where subscript $h$ means for row $h$. So finally we derived the following expression,

$$\frac{1}{2}(E(y|\theta)-E(x|\theta)) = \delta_h * (\theta y)_h - 1/4 * diag(\theta )_h$$

which says that for any two different states that differs at exactly one bit location, say $h$, we can find the difference of the energy of these two states by looking the row $h$ of 

$$\delta * \theta y - 1/4 * diag(\theta )$$

This is a useful result as for every $y \in \mathcal{D}$, we would approximate the states that have a one bit difference to be not in $\mathcal{D}$ which simplifies the computation of the cost function. Hence the cost contributed for each $y \in \mathcal{D}$, 

$$\sum_{h=1}^{n}\exp\bigg\{\delta_h * (\theta y)_h - 1/4 * diag(\theta )_h\bigg\}$$

and we have the cost function of MPF to be

$$ K(\theta) \approx \frac{1}{|\mathcal{D}|}\sum_{y\in \mathcal{D}}\sum_{h=1}^{n}\exp\bigg\{\delta_h * (\theta y)_h - 1/4 * diag(\theta )_h\bigg\}$$

The gradient of the cost function with respect to the $\theta$ matrix is

$$ \frac{\partial K(\theta)}{\partial \theta_{ij}} = \begin{cases}\delta_iy_jk_i+\delta_jy_ik_j & i \neq j\\ \left(\delta_iy_i-\frac{1}{4}\right)k_i & i = j\\ \end{cases}$$

where $k_h = \exp\bigg\{\delta_h * (\theta y)_h - 1/4 * diag(\theta)_h\bigg\}$

## Things below here can be deleted away

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import theano
# import theano.tensor as T


import os
import timeit
from datetime import datetime
from mpfntutils import load_data


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

In [2]:
def initalizeParameters(units = 32):
    """
    Initialize an initial symmetric W matrix from a random normal distribution with mean 0 and variance 1/n.
    Inputs:
    - n: size of the Boltzmann Machine (BM).
    """
    U = np.random.normal(loc = 0, scale = 1/units, size = (units, units))
    return 0.5 * (U + U.T)

In [4]:
# old cost and gradient    

# def Kcost(x, W, temperature = 1, units = 32):
#         """
#         Returns the cost computed by using the diagonals as the bias.
#         Inputs:
#         - x: samples used to train W.
#         - W: weights between the neurons of the Boltzmann Machine (BM).
#         - n: number of neurons in the BM.
#         - temperature: keep it as 1 until cost grows too big then raise temperature.
#         """
        
        
#         cost = np.mean(np.exp(1/temperature * ((0.5 - x) * np.dot(x, W)) \
#                               - 0.25 * np.diag(W)))
               
#         D = np.dot((0.5 - x).T, x)
#         Wgrad = cost * ((D + D.T) - (D * np.eye(units)) + 0.25 * np.eye(units))
#         return cost  
#         new one starts after here
        
def Kcost(x, W, temperature = 1):

    """
    Returns the cost computed by using the diagonals as the bias.
    Inputs:
    - x: samples used to train W.
    - W: weights between the neurons of the Boltzmann Machine (BM).
    - n: number of neurons in the BM.
    - temperature: keep it as 1 until cost grows too big then raise temperature.
    """
    num_samples = x.shape[0]        
    num_units = x.shape[1]
    delta = 1/2 - x
    diag = np.diag(W)[:, None].T
    E = delta * np.dot(x, W) - .25 * diag

    cost = np.sum(np.exp(1/temperature * E)) / num_samples         
    k = np.exp(E)        
    D = np.dot((delta * k).T, x)         
    C = np.zeros((num_units,))         
    np.copyto(C, np.diag(D))                 
    np.fill_diagonal(D, 0)         
    C = C - .25 * np.sum(k, axis = 0)         
    D = D + D.T         
    np.fill_diagonal(D, C) 

    return cost, D/ num_samples

In [5]:
def params_adam_opt(grad, m = 0, v = 0, beta1 = 0.9, beta2 = 0.999):
    """
    Adaptive Moment Estimation (Adam) optimizer. Takes in parameters beta1, beta2 for adaptive 
    optimization. Returns next iteration of m and v for computing the grad update.
    
    Reference: http://arxiv.org/abs/1412.6980
    """
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)
    return m, v



In [10]:
def trainmpf(units = 32, lr = 1e-2, eps = 1e-8, n_epochs = 1000,
             batchsize = 32, temperature = 1, validate_every = 100, sample = '32-50K.npy'):
    """
    Trains parameters using MPF without using symbolic gradient.
    Inputs:
    - units: number of neurons in the BM.
    - lr: learning rate.
    - eps: parameter for Adam optimizer
    - n_epochs: number of epochs to train.
    - batchsize: size of batches for stochastic gradient descent.
    - temperature: happy to be 1 for now.
    - sample: sample used.
    
    """
    print (51 * '=')
    print (24 * '#' + 'MPF' + 24 * '#')
    print (51 * '=')
    print (str(datetime.now()))
    print ('Input size: {0}'.format(units))
    print ('Learning temperature: {0}'.format(temperature))
    print ('Learning rate: {0}'.format(lr))
    
    dataset = load_data(sample)
    n_dataset_batches = dataset.shape[0] // batchsize
    W = initalizeParameters(units = units)
    
    print ('Sample used: {0}'.format(sample))
    print ('=' * 51)
    
    start_time = timeit.default_timer()

# #     Extracting learnt parameters
#     W_learnt = np.zeros((units, units))
#     b_learnt = np.zeros((units,))
# #     Renaming to W_learnt and b_learnt
#     np.copyto(b_learnt, np.diag(W))
#     np.copyto(W_learnt, W)
#     np.fill_diagonal(W_learnt, 0)

#     Loading the original W and b
    org_W = np.load(sample[0:2] + '-' + 'W' + '.npy')
    org_b = np.load(sample[0:2] + '-' + 'b' + '.npy')
   
    
#     Tracking best parameters learnt
    best_mse = np.inf
    best_W = [None, np.inf]
    best_b = [None, np.inf]
    best_epoch = np.inf
    
    for epoch in range(n_epochs):
        c = []
        current_time = timeit.default_timer()
        for batch_index in range(n_dataset_batches):
            minibatch = dataset[batch_index * batchsize: (batch_index + 1) * batchsize]
            cost, grad = Kcost(dataset, W)
#             grad = Kgrad(minibatch, W)
#             Extracting learnt parameters
            W_learnt = np.zeros((units, units))
            b_learnt = np.zeros((units,))
#             Renaming to W_learnt and b_learnt
            np.copyto(b_learnt, np.diag(W))
            np.copyto(W_learnt, W)
            np.fill_diagonal(W_learnt, 0)
            
            m, v = params_adam_opt(grad)
            W += -lr * m / (np.sqrt(v) + eps)
            c.append(cost)
            

        mseW = np.linalg.norm(org_W - W_learnt)/ ((units**2 - units) / 2)
        mseb = np.linalg.norm(org_b - b_learnt)/ units
        mse = mseW + mseb


        if mse < best_mse:
            best_mse = mse
            best_W[0] = W_learnt  
            best_W[1] = mseW
            best_b[0] = b_learnt  
            best_b[1] = mseb
            best_cost = np.mean(c, dtype='float64')
            best_epoch = epoch

        if epoch%validate_every == 0:
            print ('Training epoch %d/%d, Cost: %f mseW: %.5f, mseb: %.5f, mse: %.5f, Time Elasped: %.2f '\
                 % (epoch, n_epochs, np.mean(c, dtype='float64'), \
                 mseW, mseb, mse,  (current_time - start_time)/60) )
    
    end_time = timeit.default_timer()
    
    training_time = end_time - start_time
    fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize=(20,10))
    fig.tight_layout()
    plt.setp(ax, xticks=np.arange(0,100,16))
    ax[0,0].plot(org_W.reshape(-1,1)[0:100], 'r')
#     ax[0,0].plot(W_learnt.reshape(-1,1)[0:100], 'b')
    ax[0,0].plot(best_W[0].reshape(-1,1)[0:100], 'g')
    ax[0,0].set_title('W')
    ax[0,0].legend(['W', 'Learnt W','Best W'])
    ax[0,1].plot(org_W.reshape(-1,1)[101:200], 'r')
#     ax[0,1].plot(W_learnt.reshape(-1,1)[101:200], 'b')
    ax[0,1].plot(best_W[0].reshape(-1,1)[101:200], 'g')
    ax[0,1].set_title('W')
    ax[0,1].legend(['W', 'Learnt W','Best W'])
    ax[1,0].plot(org_W.reshape(-1,1)[201:256], 'r')
#     ax[1,0].plot(W_learnt.reshape(-1,1)[201:256], 'b')
    ax[1,0].plot(best_W[0].reshape(-1,1)[201:256], 'g')
    ax[1,0].set_title('W')
    ax[1,0].legend(['W', 'Learnt W','Best W'])
    ax[1,1].plot(org_b.reshape(-1,1), 'r')
#     ax[1,1].plot(b_learnt.reshape(-1,1),'b')
    ax[1,1].plot(best_b[0].reshape(-1,1),'g')
    ax[1,1].set_title('b')
    ax[1,1].legend(['b', 'Learnt b','Best b'])

    
    print ('The training took %.2f minutes' % (training_time/60.))
    print ('#' * 22 + 'Results' + '#' * 22)
    print ('=' * 51)
    print ('Best mse: {0}'.format(best_mse))
    print ('Best W mse: {0}'.format(best_W[1]))
    print ('Best b mse: {0}'.format(best_b[1]))
    print ('Best epoch: {0}'.format(best_epoch))
    print ('=' * 51)


In [12]:
trainmpf(units = 16, lr = 1e-2, eps = 1e-8, n_epochs = 1000,
             batchsize = 16, temperature = 1, validate_every = 100, sample = '16-50K.npy')

########################MPF########################
2017-05-31 20:58:04.769466
Input size: 16
Learning temperature: 1
Learning rate: 0.01
Sample used: 16-50K.npy


KeyboardInterrupt: 

# Things below here are kept in cases needed

In [ ]:
flow = mpf(input = s, n = 16, temperature = 1)
cost, grad = flow.Kcost(lr = 1e-2)
print (cost)
print (grad.shape)

In [ ]:
def trainmpf(units = 16, lr = 1e-2, n_epochs = 1000,
             batchsize = 16, temperature = 1, validate_every = 100, sample = '16-50K.npy'):
    """
    Trains parameters using MPF.
    """
    
    index = T.lscalar()
    x = T.matrix('x')
    flow = mpf(input = x, n = units, temperature = temperature)
    
    cost, updates = flow.cost(lr = lr)
    
    dataset = load_data(sample)
    n_dataset_batches = dataset.get_value(borrow = True).shape[0] // batchsize

    print ('Sample used: {0}'.format(sample))
    print ('=' * 51)
    
    mpf_cost = theano.function(inputs = [index], outputs = cost, updates = updates, \
                                givens = {x: dataset[index * batchsize: (index + 1) * batchsize]})
    
    start_time = timeit.default_timer()
    
    best_mse = np.inf
    best_W = [None, np.inf]
    best_b = [None, np.inf]
    best_epoch = np.inf
    
    for epoch in range(n_epochs):
        c = []
        current_time = timeit.default_timer()
        for batch_index in range(n_dataset_batches):
            c.append(mpf_cost(batch_index))
        
        W_learnt = flow.W.get_value(borrow = True)
        b_learnt = np.zeros((units,))
        np.copyto(b_learnt, np.diag(W_learnt))                        
        np.fill_diagonal(W_learnt, 0)
        
        W = np.load(sample[0:2] + '-' + 'W' + '.npy')
        b = np.load(sample[0:2] + '-' + 'b' + '.npy')
                            
        mseW = np.linalg.norm(W - (W_learnt))/ (units**2)
        mseb = np.linalg.norm(b - b_learnt)/ units
        mse = mseW + mseb

        if mse < best_mse:
            best_mse = mse
            best_W[0] = W_learnt  
            best_W[1] = mseW
            best_b[0] = b_learnt  
            best_b[1] = mseb
            best_cost = np.mean(c, dtype='float64')
            best_epoch = epoch
        
        if epoch%validate_every == 0:
            print ('Training epoch %d/%d, Cost: %f mseW: %.5f, mseb: %.5f, mse: %.5f, Time Elasped: %.2f '\
                 % (epoch, n_epochs, np.mean(c, dtype='float64'), \
                 mseW, mseb, mse,  (current_time - start_time)/60) )
    
    end_time = timeit.default_timer()

    training_time = end_time - start_time
    fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize=(20,10))
#     fig, ax = plt.subplots(figsize=(20,10))
    fig.tight_layout()
    plt.setp(ax, xticks=np.arange(0,100,units))

    ax[0,0].plot(W.reshape(-1,1)[0:100], 'r')
    ax[0,0].plot(W_learnt.reshape(-1,1)[0:100], 'b')
    ax[0,0].plot(best_W[0].reshape(-1,1)[0:100], 'g')
    ax[0,0].set_title('W')
    ax[0,0].legend(['W', 'Learnt W','Best W'])
    ax[0,1].plot(W.reshape(-1,1)[101:200], 'r')
    ax[0,1].plot(W_learnt.reshape(-1,1)[101:200], 'b')
    ax[0,1].plot(best_W[0].reshape(-1,1)[101:200], 'g')
    ax[0,1].set_title('W')
    ax[0,1].legend(['W', 'Learnt W','Best W'])
    ax[1,0].plot(W.reshape(-1,1)[201:256], 'r')
    ax[1,0].plot(W_learnt.reshape(-1,1)[201:256], 'b')
    ax[1,0].plot(best_W[0].reshape(-1,1)[201:256], 'g')
    ax[1,0].set_title('W')
    ax[1,0].legend(['W', 'Learnt W','Best W'])
    ax[1,1].plot(b.reshape(-1,1), 'r')
    ax[1,1].plot(b_learnt.reshape(-1,1),'b')
    ax[1,1].plot(best_b[0].reshape(-1,1),'g')
    ax[1,1].set_title('b')
    ax[1,1].legend(['b', 'Learnt b','Best b'])

    
    print ('The training took %.2f minutes' % (training_time/60.))
    print ('#' * 22 + 'Results' + '#' * 22)
    print ('=' * 51)
    print ('Best mse: {0}'.format(best_mse))
    print ('Best W mse: {0}'.format(best_W[1]))
    print ('Best b mse: {0}'.format(best_b[1]))
    print ('Best epoch: {0}'.format(best_epoch))
    print ('=' * 51)

In [ ]:
class mpf(object):
    """
    Minimum probability flow with bias in the diagonals
    """
    
    def __init__(self, input, n = 32, temperature = 1,  W = None):
        
        self.n = n
        self.temperature = temperature
        U = np.random.normal(loc = 0, scale = 1/self.n, size = (self.n, self.n))

        if not W:
            W = 0.5 * (U + U.T)

        self.W = W
        self.x = input
        
    def Kcost(self, lr = 1e-2):
        """
        Returns the cost. 
        """
         
        print (51 * '=')
        print (24 * '#' + 'MPF' + 24 * '#')
        print (51 * '=')
        print (str(datetime.now()))
        print ('Input size: {0}'.format(self.n))
        print ('Learning temperature: {0}'.format(self.temperature))
        print ('Learning rate: {0}'.format(lr))
        
        cost = np.mean(np.exp(1/self.temperature * (-(self.x - 0.5) * np.dot(self.x, self.W)) \
                              - 0.25 * np.diag(self.W)))
               
        D = np.dot((self.x - 0.5).T, self.x)
        Wgrad = -cost * (D - 0.5 * np.diag(D) + 0.25 * np.eye(self.n))
        
#         Tcost = T.mean(T.exp(1/self.temperature * (-(self.x - 0.5) * T.dot(self.x, self.W)) \
#                               - 0.25 * T.diag(self.W)))
#         TWgrad = T.grad(cost, self.W)

        return cost, Wgrad

In [ ]:
s = np.array([[0],[1],[1],[1],[0],[1],[1],[1],[0],[1],[1],[1],[0],[1],[1],[1]])
s = s.T
s.shape